# WBC Policy Portal Sraper
The policies are stored behind a loging page, can be multiple pages deep,


In [1]:
from  scraper import WebScraper
from loguru import logger
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import os

class WBC_SGB(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True, ) -> None:

        logger.info(f"Scraping {depth}: {url}")
        # Skip if we have already visited the page, or if we have exceeded the depth
        # or if the url is not in the domain, or if the url is not an html/ASPX file
        if self.skip(url, depth):
            return
        
        path = urlparse(url).path
        extension = os.path.splitext(path)[-1].lower()
     
        # If aspx find the base html then extract the links and recurse without saving content
        if extension == ".aspx":
            html_urls = self.aspx_link_to_html_link(url)
            for html_url in html_urls:
                self.scrape(html_url, depth=depth, starting_page=starting_page)
            return
        
        # download all resources except starting page
        if starting_page:
            html_content = self.fetch_content(url)
        else:
            if self.search_only:
                logger.info(f"Saving {url}")           
                html_content = self.fetch_content(url)
            else:
                html_content = self.save_url(url)
        
        # We have an HTML file and  have not yet exceeded the depth 
        # find all the links in wrapper and recurse
        assert( extension == ".html")
        soup = BeautifulSoup(html_content)
        for link in soup.select('div#wrapper a'):
            if href := link.get('href'):
                if not urlparse(href).netloc:
                    href = urljoin(self.domain, href)

                self.scrape(href, depth=depth-1, starting_page=False)
        return

In [2]:
def fetch_content(url:str, cookie:str):
    ''' Get the contents of a URL using a logged in cookie'''
    session = requests.Session()

    # Send the request with the cookies
    try:
      response = session.get(url, cookies={'Cookie': cookie})
    
      if not response:
        logger.warning(f"No response for {url}")
        return ""
      if response.status_code != 200:
        logger.warning(f"Response {response.status} for {url}")
        return ""
    
    except Exception as e:
      logger.error(f"Error {repr(e)} for {url}")
      return ""
    
    return response.text

def save_url(url:str, download_path:str, cookie:str) -> str:
  '''Save the url to the download path and return the contents'''
  html_content = fetch_content(url,cookie)

  full_path =  download_path + urlparse(url).path
  directory = os.path.dirname(full_path)
  
  if not os.path.exists(directory):
    os.makedirs(directory) 

  logger.info(f"Downloading {url}")
  with open(full_path, 'w') as f:
      f.write(html_content)

  return html_content


def fetch_html_links(html_page, class_names):
    '''Find all the links in the page'''
    soup = BeautifulSoup(html_page)

    links = []
    for class_name in class_names:
        for link in soup.select('div#wrapper a'):
                    if href := link.get('href'):
                        if href.startswith("/Net/Documentum/"):
                            href = domain+href
                        if not href.endswith('.aspx'):
                            print("Not an ASPX", href)
                        links.append(href) 
    return links


In [ ]:

cache = set()
domain = r"https://creditriskpolicies.westpac.com.au"

def scrape(url:str, cookie, download_path:str, depth:int, starting_page:bool, search_only) -> None:
    # Stop if when we get to the bottom
    if depth and depth <= 0:
        logger.info(f"Bottomed out at {url}")
        return 
    
    path = urlparse(url).path
    extension = os.path.splitext(path)[-1].lower()
    
    # skip already processed urls   
    if path in cache:
        return
    
    cache.add(path)

    if not url.startswith(domain):
        logger.info(f"Skipping domain {url}")
        return
    
    # WBC uses url without an extention
    if extension == "":
        extension = ".html"
    # Skip unwanted videos etc
    if extension != ".html":
        logger.info(f"Skipping unwanted extention  {url}")
        return
    
    # download all resources except starting page
    if starting_page:
        html_content = fetch_content(url,cookie)
    else:
        if search_only:
            logger.info(f"Saving {url}")           
            html_content = fetch_content(url,cookie)
        else:
            html_content = save_url(url, download_path, cookie)

    # if we have not yet exceeded the depth find all the links in wrapper and recurse
    if extension == ".html" and depth and depth > 0:
        soup = BeautifulSoup(html_content)
        for link in soup.select('div.rn_Body a'):
            if href := link.get('href'):
                if not urlparse(href).netloc:
                    href = urljoin(domain, href)
                
                scrape(href, cookie, download_path, depth-1, starting_page=False, search_only=search_only)
    return

# St Georges

In [41]:
cookie="fpid=b923bc8a-b570-453a-8550-ca415ad93e49; TS01bae890=0197f87ceab1be61449171479bc81637a88008fb441eca7193b19f96734ae47214620180b48f3280531e473aed6c792b1be219a84f; aam_fb_ads=fb_ads%3D22915918%3B22915983%3B22969655%3B24893962; aam_gads=googads%3D16500962%3B24893962; aam_sem_gads=googsemads%3D22969655%3B24893962; AMCV_3A4B7BAF56F01DA67F000101%40AdobeOrg=MCMID|22444576802660860302750140358855840962; _gcl_au=1.1.860570339.1733966434; __qca=P0-1981608762-1733966434155; _ga=GA1.1.689035334.1733966435; LPVID=RiNzc3MWJlMzgwOGNlY2Zk; aam_edge_seg=segment%3D12324502%2Csegment%3D19586959%2Csegment%3D22915918%2Csegment%3D19586952%2Csegment%3D22969655%2Csegment%3D13955774%2Csegment%3D12324503%2Csegment%3D18159756%2Csegment%3D22915983%2Csegment%3D24893962%2Csegment%3D11996985%2Csegment%3D11996985; ADRUM=s=1734322539998&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fanswers%2Fanswer_view%2Fa_id%2F1000724%3Fhash%3D1150; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_identity=CiYyMjQ0NDU3NjgwMjY2MDg2MDMwMjc1MDE0MDM1ODg1NTg0MDk2MlITCIGc0MO7MhABGAEqBEFVUzMwAPAB2OSslr0y; s_ecid=MCMID%7C22444576802660860302750140358855840962; policyType=MORTGAGE_BROKING_OBP_SGB_GROUP; LPSID-16303569=UWxoN8jOTpObjrbqfnWhEg; _ga_WQCDREZ4DH=GS1.1.1734412272.6.1.1734412610.0.0.0; cp_profile=eUD1rOyVv1i2dRn0X9NfJZzNfKXaRdYJH3oqkb7QBfgshGxInFyXi4e_JOHbes58H7JtREzb2J0pFTHLv7zTL2ikYZGzNy9a6PRQsurRfC8o4BkLfvx29NuwFulVmgdyx0DA13Yu3u4Mpmh7e8op6RtRciIFIiNbJ2cJY96lnsKjqAUVX6sTpHMw!!; cp_session=fUi~TUtQW8BPN6mIsoml3w5rgzKcro5ObseK0IcgMvUc1ayC9~7~D8ajqUC~S~9KnsFV3TFHmj~wMWXG7VQLwNBqb6l7DQSh5Id3XPvRGaM54q09XF2yJJAEx~xNDjtZd8qcZ0NWP54a7oo1Hq_opHDMc2Ncxq1X1eHsttMGjv_cHNzroTklfuhCNQ5rs7O2dUZlgc_wuWnJW588Tb3GuX5bCWgT~DJh8u5J9AuIAbV1sG10MZp_ezTZk79YKJxi6B7lTQ7~uOh5j3XGG8ZPh9Pn580AwKlnJmvGjeJRPujEn1KVrAgYGIVwGFdjQJD6STHj5RKNXrdo43_gQWKgl0xNdo~K~jELHWOC8g0HXnT6VxhG6xhYU7UwPCLPL~AHqxYC12Q0GSar9FVHO~6OpamjgAdCPTNAWbIgUvkhMgz2CysKjEKf6Qz5G108k5tbgv9HTISMkpUdNysl1ZpQ9ECKJ4E8quX1mCeg2Ebi5CWqiQ_k1CZEnM5tBtBuh32VoqmMmBgF2A199g7V2EySOqK78MykYL1e3cn_9qs5bfmWzqchwXUHrMzku3QLjK9cfH_qVyEQldA4DizrVYZsCrRP40m4Zeo7hoxecnpV2Uax5lEG9Oz3JYWK9R9iEi0XtflMLtuVf11XsfRxpFC__rBy7XtAa9othqgvL3olWhlY5v6WGjO7TWmt3zAP1dIj41~igKww1NVXQXGyrQWOSekxLbo2BT1TzlDCGzUCpgd1V_QdEu9Mem6vZuxbt3RZ~IebG9YYTSYBYnr5klNICgl8CRCszr~ZlEvl~d7IegkDho5weLNhvVfAXKcRmkNfdjxyK70onh7~qgA0iEPMUT4SKqPSnztu9IEFowdzP71imgE3iwRQ6vv_Vhg02UOpEOkMmA1ANI2ipN6~Ovc71vMBW9P1SN2HbDydS_Q3V3~Gi5lvCqEovMVCYsccZJPWIEuxCJZkR6afqBlvS8UlJJxr8ChIl2KgQQTuwoVOyAzYUdshr2JXIKiRK89XP8vv91L_TXlO38F1IHnGN7NwZCOAnb85jmPnbKjcSSw_gMaXAA_CcyKCDKnTEBz4hMmRwb4TYQKDYl104nPioQ8xJHxMxS9~nrXjM6n~MdblHx~GB~T~eKJ_B4JcEYJGACmnYlB5BMJZwE9OoYBXI_Mvedj_97SRPQiLgxC3L2edGkFhndvHouUmnT_nYImkh0C8eSIe3Sw1Y~bpW_3gk3CztHXezxD~Y0WBES1dlNDkVOkDcLK4kF4~q~4zxgYUWNFXHwtgt59zHR8Z0nNPja0huHDNucUQ1pgZfd_OWa0iXnnGyJ~zD8mzmNfDPipDF8cLM~Hu09QjyI7XQalTU9BOzzidwRn1mp1So8Qi73VxNcd_Ipw7g_WzjiXt4RIZFjiLwMmcz4Fyvo1Eh5BgQRru6gVziGU8rfOAdJhopJny52HRpcE0HpS0Ny9HwhelMEsYklZ_Crt_8EmwsV44K6VQsIdYVjvy6mWJ9vIU1jvHKyCaeJ2trsnTmBF3~yvRJlCLFWd7Q8NupCQKVpnm4Sf2~1cHTufs1vqCN8S5t899vU2NIWBDTMXYhD2uQz1aTns2OSJAaDXqkneHZQf4C5GBCFdsgDypyF68eRBgZmYhICW2aT9kpL3T9YRg_oyJvx0m8a_yG6~Nw8b9X6HP8papYA8qKC4V3engzZzc8Ud9q4n9LAZnYHzTRxCLJKNEWzgDfvsb~8x11o0alNoLO_Md4qL_4fVgcog5JSdcoW7AOH6vDoVbdhfeDv6CA010GkyobNjQkGU19ToSdLwC5wU65ENK~hByopsI5Lr1Pg8p6jcJe_thU4naN0IyiUAqNSHEZINOEFD3Nez8yeDGp1SEwZvaW~gSOVEniBr51pV2KAB7JZirdIKWMVC_9TyE8hQTI~iiCkHrivc91hmHzDUJLtxRYBu3EjSA7PgtjZWwGtjMZ~6xv8HfVxR~AdisVW2VmcJG~cgNsp3s2EEzGqcYaC8clEgfmr7I~nRP9gZFVPc7ObncYXqa_ld2tmelLHeXcrFOY7mj~ifcu3x~Uehb2TqEuMde2RbrKPe_AiZIa5ttm1pQlPiboyzQcAti4XXUb87qmt9yav4EPjVa3rjp3UOqHvHzNBl7RHFxVfBWdIge4hNj0quSAg7XWVPI9i9tA1fSS8SQYEqasfvUFKtnqZImR34Y~G9Q_wPQluhhev9k0CRvdj9ojHcjHKSz6EGiHal4Nn0Rk61XUh9zXsgXE0kLITPLvDkr0iOVUhpbptZQrll7uZJJ_QSO5VGZBKpitYeivlg5n_0MV5t7z96Ra26ag3KOYHfiv1SMyAhnXBsk2bYIZqjTDlPoZWLSLP4lbXrtSMP2T1Dcw7LosILHjjmPAmaGQ4MUWUKIiNckGgs6nfDb23fRmCQIzEV1GXdtD0ohBiSgN5NPE9VOzVxnTp65CPhC3bRLrDM~haftJWg92q1Tp~671iFjiDeN9JHE3XmfCb9FTFH1n6B4zECx7gzP6v45vTDH2AB0wWwWqpVgCBXrxgpL~YRsLVaV14CDygzmmX8whuapaBrrDMp683JNLZ5OFtyf8vv7Bpamm_SGl8YISIQhmmjHpFORZDzW6TowZra7oqL1bu1y0vHb6TMISNR7iqjJ1OxKq~QtxtUzLUtxe5OP_Jd2Gx26htw3IGtT8TpA7kRgs01yc4HDDpHaqt~1eT3aHNDoSdwBm~hwlqQtAvg5_pyHxkX~lsgk7m4WDMvtlaKcm2pOpwon3ga6~LTPTh5iLfG00PcNk9yRyyDpQ2UEzfduGblbyZNWsDHf3E1TiPBZLJuC8~umrDlpQxvX~lolevaTf4b3cKej_b9iIPr42PkkXtdCOpGHsdhDVNhgP9ptUprMTD_yKR_N1og7gl290BaP2O4VBwM4Jj9AREy5ppdKxn6LYx_NDds9jiOaWYxg7n4dHBvPjAzeeVo1vMGrdW6jAoEeW7hJhXNJRyQdxLhjykPMYhEwyNNDh_5ggnLuvFO1Gi~7clJyPRJCUEef7cPzSfmnciKEO1nP8oZ8sUNc0Z6gb3Z543TtRO~uQtFqn~ejFeWQ6AMUPtANrHugaKCD3g9YwozBdrtJMYgarO26kU1Syh4Cq4nKJil70nn4YtwM_nzxckYNVQwkMuYiQeEsUPvOjwKKtxEOOLdGbblyEaE6s3Gv96nt8d1fFH_tkzXodvvGpGFHkV8XDW2FG2xuMsMEia~NO2q~P812Cq9WQmennVnZhY9ly3fVlYJKlkOUF7CbzhANr08hitpWO8HKzI~eCopnx3xXfSq2rZPz91O5EA43wGPMumZntw2vZuDQl2YY8sSotopuZ8RSoaNo2Lwq7k74JOvqOMW_C~at_wnHtfGS5dYuR3hGloB0XRaqwThGjnuxHkYhGWGJ6mPRdFNgI179nmp~Rxa9kieDTOWGPNlrjYG7WDWZPKHt_ePOdJnG0xCNRDkoEX7XKxfg6YbbOPGHamduiFHFXniQQ9EuQMG_xVPH1MJXyM7rJ~YbEXg7Q7gUZBJ04h78SMAxKHjg7OGoLnqDeWgVPl46N_zxQrzolNeVVlC8by0Xji1EfQL6Q5QpNb_SDeI_4Mojd3CWQuTMOgL~zfK5LBytrBLNe~UEuqQK_H6laPWFI94twfh9i27Q5YmEDXtf1PHwuMyBxwsm6WdQZbkrl1qYcvZbj4fJTmKQQAbvO55m9QipiS~GI7zu2~co3ecYCQ2SIVS6p37PljjrsAlBslYGnITFIccE8mahikx_Htmjlqr_J6QPXy9pmMkyv929BDkYyPguBoQeHT7kLWnNRlWro~qDJrhXAdA2bi_aLJ2SsJjlRVbmYpAgpge0cOciAQVbWSQKDOHajcEtx0i1rUY15BsHrHVmQRlJvlhHDAvpTfzwnxqHLPcp5hbYmI6OjU3pBI9fZlIYbOopKDi~dKMAL8Gkg_G7Fv7st1AqvmhzQ!; ADRUM=s=1734414649212&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fbrowse%2FchannelRecordID%2FMORTGAGE_BROKING_OBP_SGB_GROUP%2Fa_status%2Fpublished%2Ftype%2Fbrowse"
directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cache = set()
scrape("https://creditriskpolicies.westpac.com.au/app/browse/a_status/published/channelRecordID/MORTGAGE_BROKING_OBP_SGB_GROUP" ,    
       cookie, depth=2, download_path=directory_path, starting_page=True)

2024-12-17 14:01:26.718 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000681 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies//app/answers/answer_view/a_id/1000681.html
2024-12-17 14:01:28.100 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/results/kw/BOS1/nlpsearch/true/loc/en-GB to /home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies//app/results/kw/BOS1/nlpsearch/true/loc/en-GB.html
2024-12-17 14:01:29.177 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000726 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies//app/answers/answer_view/a_id/1000726.html
2024-12-17 14:01:30.929 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000781 to /home/ke

# Westpac

In [9]:
cookie="fpid=b923bc8a-b570-453a-8550-ca415ad93e49; TS01bae890=0197f87ceab1be61449171479bc81637a88008fb441eca7193b19f96734ae47214620180b48f3280531e473aed6c792b1be219a84f; aam_fb_ads=fb_ads%3D22915918%3B22915983%3B22969655%3B24893962; aam_gads=googads%3D16500962%3B24893962; aam_sem_gads=googsemads%3D22969655%3B24893962; AMCV_3A4B7BAF56F01DA67F000101%40AdobeOrg=MCMID|22444576802660860302750140358855840962; _gcl_au=1.1.860570339.1733966434; __qca=P0-1981608762-1733966434155; _ga=GA1.1.689035334.1733966435; LPVID=RiNzc3MWJlMzgwOGNlY2Zk; aam_edge_seg=segment%3D12324502%2Csegment%3D19586959%2Csegment%3D22915918%2Csegment%3D19586952%2Csegment%3D22969655%2Csegment%3D13955774%2Csegment%3D12324503%2Csegment%3D18159756%2Csegment%3D22915983%2Csegment%3D24893962%2Csegment%3D11996985%2Csegment%3D11996985; ADRUM=s=1734322539998&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fanswers%2Fanswer_view%2Fa_id%2F1000724%3Fhash%3D1150; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_identity=CiYyMjQ0NDU3NjgwMjY2MDg2MDMwMjc1MDE0MDM1ODg1NTg0MDk2MlITCIGc0MO7MhABGAEqBEFVUzMwAPAB2OSslr0y; s_ecid=MCMID%7C22444576802660860302750140358855840962; policyType=MORTGAGE_BROKING_OBP_WESTPAC; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_cluster=aus3; LPSID-16303569=jpjdXxtuR0mpuS1bw3Q_-A; _ga_WQCDREZ4DH=GS1.1.1734415504.7.0.1734415507.0.0.0; cp_profile=eU6pvc5N0qMYpUtWSz_YmEBrjjE6V9y6BKtjIJR~5c_ydzIRPhOD47TTfuiy6rS_iQMAbCChTAnhk3lTf_6ipJp5FJS37jGj1c10~7nDLADKZen0ypgaGui5oc1om_0LFtBT0Il0KXKzESdZHcz2Ohj32yya2oOb5j; cp_session=fUb4rL_K1UBE8K3e_rnX9s78Hx~AASQU8ejXuEqsia~m7SjSkduE2T3_hGYQLeBV_0QzSzm~sLXgwOk2lls~gM1MI3gAB43OFB4zcumdahJHlKKstoOqj4AYIn26WvvfiQgJadHd~AIvTo_P8Y6jNnJXpctYHNp2DVRHN9gdi473wDf2WEcrfnWeB0kuEqwpaNNFENG65S1XH7ic5VDs_l7_KTDi3X8rYhwwni4oVDWGqmiYLdkt0DPxtTuCSQ8u0WpLKVa63Bxmq1Y4ME19vBy0BvdGf_hPfkfziBTPeQ0v3SpzX391ujYOF45vTaUOx62VYUV3bWW5SKtCMG23luChg6dS9kbJRBQaVZvj6nCn~gWncE7fL31ZRsixf1lqdNtql9h9T~tixE5BJU1ag0alvdrxuFgtl9V2WZOYxwIv9gevuMAAU6ipNPtvijyHwzBThZOpst53q2QrUJOrBHP_Te3idLHvy1xWnmXXevjUt0LEvE9pGNqNlH1qxO4gRmoflWZ8bLPJmrraeTHiTDRdbfPxBbcpwOLusZLsz1RVL0J8tyt0SWXYjHd33wIrZlFqmTYkLUwDgXwtjHagIh6i2gP2J2mcEM9Yb4GtTWiAFpvL2gOy~2k67SrpCTk_J8hIOSN4PBSH3aOJwSvBAeRVCN7dqEduATTE7SKj7103~HZwCoDgvhJGWoh4mDCcKmYA5t1v5vdPYkfmFRYNJAhyL8UwdY7dUU2OBkvZxCIm~ki5UA5UqVKTspQUUK6lKuVkWzB99UlpShLusNNIKsmiAswfb0ZNxWERu4~Yca0FnZ735A7EjKdr1bnBo6SnImIaJM~1B4YGf8Orp3qM1Xy38EWHofMX_dhE2FtJWHTfGji7Kj9BZu97lWisCK7BEDmMNLrrA~2MU3OrxuzXGxOlVVr7IKPFS3AKePCQRh4~jgLp749_uIAIrRoPJ0_Y4Y7oOLs2rIojXigjB3tchIWw1~RudgtW4zCHLAk3Ny3cdvF5dwu14~gck5z1BzqbNd2k1mSMHy0iS8UdbvDA3SRqZho71nGzZC8y8eOHtahfkflQmeFrSAuTTzIWMIey~wL2NgUW5EabdHqf6CMkuj0__nq3mILM~Q~atHBdeuMTlIsCzWgmPR9wRJBl243RvTEV67ypvQOUkeUB9TtGisUIBEOq0HT_X~kAc_V9YUmpmbEZFLe574LnmDcoo03jODHumXcE1ubBn8r_6JMp3eJt~Fj3YNJD9HsHaaYJnefK2atvhLb7H_YIPfsLUG6LwFy~W3pF2cU0Ni1lSLrsQbjfHFvWObTqyxCORN42i8oLef42J4Z~EaqMK2mpwNelaln04lLSZ~0C7HaDdMQu6L8_ILE7f4UB4khYZb6rPaz6goOwdDMW4CSoIyqsrKfM53HA7ypfyBcLjU7o49n9tuQRJGI_1vf8jwo_Deg4UyPmwdXlFyiwOUJw!!; ADRUM=s=1734415538585&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fbrowse%2FchannelRecordID%2FMORTGAGE_BROKING_OBP_WESTPAC%2Fa_status%2Fpublished%2Ftype%2Fbrowse"
directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cache = set()
scrape("https://creditriskpolicies.westpac.com.au/app/browse/channelRecordID/MORTGAGE_BROKING_OBP_WESTPAC/a_status/published/type/browse" ,    
       cookie, depth=2, download_path=directory_path, starting_page=True)

2024-12-17 14:09:20.124 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000680 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies//app/answers/answer_view/a_id/1000680.html
2024-12-17 14:09:21.359 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/results/kw/BOW1/nlpsearch/true/loc/en-GB to /home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies//app/results/kw/BOW1/nlpsearch/true/loc/en-GB.html
2024-12-17 14:09:22.410 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000725 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies//app/answers/answer_view/a_id/1000725.html
2024-12-17 14:09:23.365 | INFO     | __main__:save_url:40 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000782 to /home/ke